# Train model using tensorflow embed data 

In [1]:
# import lib
import numpy as np
import json
import pandas as pd
import os
from sklearn import svm

In [2]:
# df = pd.read_csv("DCForTrain.csv")
# df.head()

In [3]:
path = 'cleanedDCinboxResource/'
dest_path = 'encodedDCinboxResource/'
dirs = os.listdir(path)
dirs
topics = map(lambda x: x.replace('.json.csv',''), dirs) 
topics = list(topics)
topics_df = pd.DataFrame(topics)
topics_index = list(topics_df.index)
topics_index

topic_dict = dict(zip(topics, topics_index))
topic_dict

{'healthcare': 0,
 'pet': 1,
 'business': 2,
 'fashion beauty': 3,
 'fitness sports': 4,
 'food beverage': 5,
 'media enterainment': 6,
 'lifecycle home': 7,
 'consumer': 8,
 'green cleantech': 9,
 'children education': 10,
 'software tech': 11}

In [4]:
# Set up training data
df_train = pd.DataFrame()

for i in dirs:
    temp = pd.read_csv("encodedDCinboxResource/" + i)
    temp['topic'] = i.replace('.json.csv','')
    df_train = df_train.append(temp.iloc[:temp.index.size//2,:])

In [5]:
# Set up for X_train
X_train = df_train[['embed']]
X_train['embed'] = X_train['embed'].apply(json.loads)
# X_train['embed']
X_train = pd.DataFrame(X_train['embed'].tolist(), columns=range(0, 512))


from sklearn.decomposition import TruncatedSVD

X_reduced = TruncatedSVD(n_components = 50, random_state = 0).fit_transform(X_train)
X_train = pd.DataFrame(X_reduced, columns=range(0,50))
# X_train = X_train.sample(frac=1,random_state = 5)


# X_train = df.copy(deep=True) # copy the dataframe 
# X_train = X_train[['embed']]
# X_train = X_train.iloc[:22981,:]
# X_train['embed'] = X_train['embed'].apply(json.loads)
# # X_train['embed']
# X_train = pd.DataFrame(X_train['embed'].tolist(), columns=range(0, 512))
# X_train.head(2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
# Set up for y_train
y_train = df_train.copy(deep=True)
y_train = y_train[['topic']]
y_train = y_train['topic'].map(topic_dict)
y_train = y_train.reset_index(drop = True)
# y_train = y_train.sample(frac=1,random_state = 5)
y_train


0         0
1         0
2         0
3         0
4         0
         ..
22839    11
22840    11
22841    11
22842    11
22843    11
Name: topic, Length: 22844, dtype: int64

In [7]:
# settinf up model

clf = svm.SVC()
clf.fit(X = X_train, y = y_train)
# dump the model for further usage
from joblib import dump, load
dump(clf, 'tweetModel.joblib') 


['tweetModel.joblib']

In [8]:
df_test = pd.DataFrame()

for i in dirs:
    temp = pd.read_csv("encodedDCinboxResource/" + i)
    temp['topic'] = i.replace('.json.csv','')
    df_test = df_test.append(temp.iloc[temp.index.size//2:,:])


    
# Set up for X_test
X_test = df_test[['embed']]
X_test['embed'] = X_test['embed'].apply(json.loads)
# X_train['embed']
X_test= pd.DataFrame(X_test['embed'].tolist(), columns=range(0, 512))
X_test_reduced = TruncatedSVD(n_components = 50, random_state = 0).fit_transform(X_test)
X_test = pd.DataFrame(X_test_reduced, columns=range(0,50))
X_test

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.433665,-0.300330,-0.258646,-0.006743,0.040009,-0.062847,-0.088324,-0.085778,0.143490,-0.072497,...,-0.045573,0.036898,-0.091726,0.026516,0.011907,-0.027616,-0.107099,0.013576,0.079907,-0.034356
1,0.031923,-0.025417,0.014876,0.088494,-0.016803,-0.205721,0.001168,-0.010152,0.120703,-0.066450,...,0.030698,-0.044277,0.010753,0.100594,-0.025907,-0.007513,0.160023,0.132531,-0.047516,0.066379
2,0.538038,-0.185097,-0.331686,0.002332,0.110980,-0.012044,-0.028951,0.010553,0.048111,-0.110087,...,-0.106713,-0.128402,0.053386,0.121790,0.006403,0.042294,0.075902,-0.029979,0.001999,-0.014662
3,0.164621,0.048707,-0.160095,0.118009,-0.119589,0.013265,-0.037521,0.267212,0.013528,-0.136600,...,-0.062009,-0.023080,-0.003976,0.002577,-0.060106,0.040134,-0.028644,0.093866,0.018757,0.007569
4,0.134802,0.102678,-0.073790,0.166910,0.001184,0.059679,-0.258499,0.201932,-0.092750,0.179935,...,0.038618,0.069579,-0.032078,-0.009885,0.103096,-0.083153,-0.054665,0.010873,-0.014237,-0.099475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22844,-0.043832,0.055413,0.082503,-0.102701,-0.001854,0.134199,0.080034,-0.118629,-0.064628,0.073833,...,-0.026067,0.060851,0.058407,0.025040,0.140596,0.023984,0.017319,0.048367,0.067492,0.072318
22845,0.088654,0.060587,0.128061,-0.152659,0.038111,0.245061,-0.031752,0.111618,0.219069,0.041505,...,-0.018163,0.147643,-0.076512,0.073008,0.154606,-0.042940,-0.067832,0.167693,0.070880,0.086594
22846,0.199693,0.110279,0.027263,-0.273061,-0.117996,0.212826,-0.090640,-0.157041,0.061467,-0.013568,...,0.076798,0.020782,0.005607,0.027434,0.143662,0.020939,-0.021284,-0.008084,0.051726,0.008690
22847,0.271188,0.143580,0.143182,0.048309,-0.241884,0.245663,0.071801,-0.292509,-0.001896,-0.064961,...,0.038891,0.136189,-0.088013,-0.003500,0.216560,0.124781,-0.017238,0.051184,0.044711,0.005829


In [9]:
y_test = df_test.copy(deep=True)
y_test = y_test[['topic']]
y_test = y_test['topic'].map(topic_dict)
y_test = y_test.reset_index(drop = True)
y_test

0         0
1         0
2         0
3         0
4         0
         ..
22844    11
22845    11
22846    11
22847    11
22848    11
Name: topic, Length: 22849, dtype: int64

In [10]:
Y_predict = clf.predict(X_train.sort_index())

output=pd.DataFrame(data={"y_train": y_train.sort_index(), "Y_predict":Y_predict}) 

output.to_csv(path_or_buf="resultssvm.csv",index=False,quoting=3,sep=',')

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline

# pipeline and fit to training set
pipeline = make_pipeline(
    StandardScaler(),
    svm.SVC()
)
pipeline.fit(X=X_train, y=y_train)
    
scores = cross_val_score(pipeline, 
                         X=X_train.sort_index(),
                         y=y_train.sort_index(),
                         cv=5)  # cv refers to number of cross-validation cuts
scores

array([0.07616546, 0.05821843, 0.08732764, 0.09848982, 0.11011384])

In [17]:
Y_predict = pipeline.predict(X=X_train.sort_index())
from sklearn.metrics import accuracy_score
accuracy_score(y_train.sort_index(),Y_predict)
output=pd.DataFrame(data={"y_test": y_train.sort_index(), "Y_predict":Y_predict}) 

output.to_csv(path_or_buf="results.csv",index=False,quoting=3,sep=',')

0.35711784275958675

In [13]:
dump(pipeline, 'tweetModelCVPipeline.joblib') 

['tweetModelCVPipeline.joblib']

In [18]:
Y_test_perdict = pipeline.predict(X=X_test)

accuracy_score(y_test,Y_test_perdict)
output=pd.DataFrame(data={"y_test": y_test, "Y_predict":Y_test_perdict}) 

output.to_csv(path_or_buf="results_test.csv",index=False,quoting=3,sep=',')

0.08744365180095409